In [1]:
import torch

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\code\datasets\riceClassification.csv")

In [4]:
df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [5]:
df.describe(include='all')

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
count,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000
mean,9093.000000,7036.492989,151.680754,59.807851,0.915406,7225.817872,94.132952,0.616653,351.606949,0.707998,2.599081,0.549079
std,5249.701658,1467.197150,12.376402,10.061653,0.030575,1502.006571,9.906250,0.104389,29.500620,0.067310,0.434836,0.497599
min,1.000000,2522.000000,74.133114,34.409894,0.676647,2579.000000,56.666658,0.383239,197.015000,0.174590,1.358128,0.000000
25%,4547.000000,5962.000000,145.675910,51.393151,0.891617,6125.000000,87.126656,0.538530,333.990000,0.650962,2.208527,0.000000
50%,9093.000000,6660.000000,153.883750,55.724288,0.923259,6843.000000,92.085696,0.601194,353.088000,0.701941,2.602966,1.000000
75%,13639.000000,8423.000000,160.056214,70.156593,0.941372,8645.000000,103.559146,0.695664,373.003000,0.769280,2.964101,1.000000
max,18185.000000,10210.000000,183.211434,82.550762,0.966774,11008.000000,114.016559,0.886573,508.511000,0.904748,3.911845,1.000000


In [6]:
df[df.isna().any(axis=1)]

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class


In [36]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
x_colums=columns=['Area','MajorAxisLength','MinorAxisLength','Eccentricity','ConvexArea','EquivDiameter','Extent','Perimeter','Roundness','AspectRation']
scaler = MinMaxScaler()
X=pd.DataFrame(scaler.fit_transform(df[x_colums]),columns=x_colums)
Y=df['Class']

In [37]:
X.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation
0,0.262097,0.165901,0.614922,0.149139,0.248903,0.337191,0.544960,0.244209,0.807934,0.032371
1,0.045525,0.005123,0.352934,0.168567,0.051726,0.066336,0.655171,0.036283,0.899898,0.037204
2,0.068418,0.019803,0.366292,0.188069,0.065607,0.098164,0.746848,0.041724,0.950265,0.042222
3,0.071670,0.026591,0.363903,0.213671,0.068573,0.102609,0.795277,0.043795,0.952688,0.049077
4,0.152315,0.100769,0.456247,0.250355,0.145094,0.207586,0.767156,0.106958,0.958906,0.059471


In [38]:
import numpy as np
xarray=np.array(X)
yarray=np.array(Y)

In [39]:
yarray

array([1, 1, 1, ..., 0, 0, 0])

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(xarray, yarray, test_size=0.2, random_state=42)
X_val,X_test,y_val,y_test=train_test_split(X_test,y_test, test_size=0.5,random_state=42)

In [41]:
print(X_test.shape,X_train.shape,X_val.shape)

(1819, 10) (14548, 10) (1818, 10)


In [42]:
from torch.utils.data import DataLoader, Dataset

class dataset(Dataset):
    def __init__(self,X,Y):
        self.X=torch.tensor(X, dtype=torch.float32)
        self.Y=torch.tensor(Y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index],self.Y[index]

In [43]:
y_test

array([0, 1, 1, ..., 1, 0, 1])

In [44]:
train_data=dataset(X=X_train,Y=y_train)
test_data=dataset(X=X_test,Y=y_test)
val_data=dataset(X=X_val,Y=y_val)

In [45]:
train_datalaoder= DataLoader(train_data, batch_size= 8, shuffle=True)
test_dataloader= DataLoader(test_data, batch_size= 8,shuffle=True)
val_dataloader=DataLoader(val_data,batch_size= 8,shuffle=True)

In [52]:
import torch.nn as nn
 

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel,self).__init__()

        self.input_layer=nn.Linear(X.shape[1],20)#colums are given by shape[1] and the other input is no. of nodes in hiddenl layer
        self.linear=nn.Linear(20,1)#20 for hidden layer and 1 for ouput layer
        self.relu=nn.Softmax()
        # self.relu=nn.ReLU()
    def forward(self,x):
        x=self.input_layer(x)
        x=self.linear(x)
        x=self.relu(x)
        return x
    
model=MyModel()
        

In [47]:
from torchsummary import summary

summary(model(train_data.shape[1],))

AttributeError: 'dataset' object has no attribute 'shape'

In [53]:
import torch.optim as optim
critetion=nn.BCELoss()
optimizer=optim.Adam(model.parameters(),lr=1e-3)

In [57]:
total_loss_train_plot=[]
total_loss_val_plot=[]
total_acc_train_plot=[]
total_acc_val_plot=[]

for epoc in range(10):
    total_acc_train=0
    total_loss_train=0
    total_acc_val=0
    total_loss_val=0

    for index in train_datalaoder:
        input,lables=index
        prediction=model(input).squeeze(1)#canges the dimestion of the data from 2D to 1D
        batch_loss=critetion(prediction,lables)
        
        total_loss_train+=batch_loss.item()
        
        acc=(prediction.round()==lables).sum().item()

        total_acc_train+=acc

        batch_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    with torch.no_grad():
        for index in val_dataloader:
            input,lables=index

            prediction=model(input).squeeze(1)#canges the dimestion of the data from 2D to 1D
            batch_loss=critetion(prediction,lables)
            
            total_loss_val+=batch_loss.item()
            
            acc=(prediction.round()==lables).sum().item()

            total_acc_val+=acc

    total_loss_train_plot.append(round(total_loss_train/1000,4))
    total_loss_val_plot.append(round(total_loss_val/1000,4))
    total_acc_train_plot.append(round(total_acc_train/train_data.__len__()*100,4))
    total_acc_val_plot.append(round(total_acc_val/val_data.__len__()*100,4))

    print(f"Epoch:{epoc+1},train_loss:{round(total_loss_train/1000,4)},Train accuracy{round(total_acc_train/train_data.__len__()*100,4)}")
    print("="*20)



Epoch:1,train_loss:82.1375,Train accuracy54.8529
Epoch:2,train_loss:82.125,Train accuracy54.8529
Epoch:3,train_loss:82.1,Train accuracy54.8529
Epoch:4,train_loss:82.1125,Train accuracy54.8529
Epoch:5,train_loss:82.1375,Train accuracy54.8529
Epoch:6,train_loss:82.15,Train accuracy54.8529
Epoch:7,train_loss:82.1375,Train accuracy54.8529
Epoch:8,train_loss:82.1125,Train accuracy54.8529
Epoch:9,train_loss:82.1125,Train accuracy54.8529
Epoch:10,train_loss:82.1,Train accuracy54.8529


In [ ]:
with torch.no_grad():
    total_test_loss=0
    total_test_acc=0
    for data in test_dataloader:
        input,lables=data
        prediction=model(input).squeeze(1)
        batch_loss_test=critetion(prediction,lables).items()
        total_loss_test+=batch_loss_test
        acc=(prediction.round()==lables).sum().item()
        total_acc_test+=acc
    print(f"Epoch:{epoc+1},train_loss:{round(total_loss_train/1000,4)},Train accuracy{round(total_acc_train/train_data.__len__()*100,4)}")
    print("="*20)    
